# Question 1

In [1]:
using Distributions
using Random
using LinearAlgebra

In [2]:
beta = [1, 2, 0,0,0,0,0,0,0];

In [38]:
# sample data creation
Random.seed!(123)
n = 50
std = 0.5
du = Uniform()
Xvar = rand(du, n, 8)
constant = ones(n)
X = hcat(constant, Xvar*2)

dn = Normal()
eps = rand(dn, n)

y = X*beta + std*eps;

In [50]:
# set prior
m = zeros(9)
C = zeros((9,9)) + I
a = 1.0
b = 1.0;

In [51]:
# MCMC
ns = 1000
burn= 0.1*ns
betaresult = zeros((9, ns))
phiresult = zeros(ns)

# initialize
b0 = m
phi0 = a/b
phiresult[1] = phi0

# loop
# MvNormalを使うとできないのでcanonical formを使う
posa = n/2 + a

for i in 1:ns
    
    if i == 1
        sigmainv = phiresult[i]*X'X + inv(C)
        h = phiresult[i]*X'y + m
    else
        sigmainv = phiresult[i-1]*X'X + inv(C)
        h = phiresult[i-1]*X'y + m
    end
    cbeta = rand(MvNormalCanon(h, sigmainv))
    betaresult[:,i] = cbeta
    res = y-X*cbeta
    cphi = rand(Gamma(posa, dot(res,res)/2 + b))
    phiresult[i] = cphi
end

    

In [52]:
# phiはGammaじゃダメなんだっけ？
phiresult

1000-element Array{Float64,1}:
 302.8093344396621 
 220.53770529029825
 229.67033558767253
 234.195393845848  
 209.42511841921294
 135.72903922390478
 228.89520415619666
 192.14442485636906
 198.60429760082428
 164.96109048217414
 224.57995457592833
 209.72280556760748
 208.32164289153508
   ⋮               
 192.18026908326758
 280.1668446120746 
 194.15768902164027
 104.00179209594496
 221.92023503668793
 201.046925371117  
 236.08249127263662
 166.55259993074424
 146.46747931255433
 146.2666658554907 
 191.87819897774705
 172.94787969654598

In [53]:
# betaはうまくいってる
betaresult

9×1000 Array{Float64,2}:
  1.17169     1.09845      1.14667     1.09418    …   1.05121      1.12355   
  2.7032      2.03363      2.03908     2.05027        2.02645      2.07033   
  0.185696    0.00837036   0.0146247   0.015685       0.018322    -0.00642985
 -0.554951   -0.0633411   -0.0776021  -0.0795953     -0.0866694   -0.0710032 
 -0.373799    0.016822     0.0370754   0.030997       0.0425688   -0.00564404
  0.164809    0.212708     0.206972    0.256104   …   0.279801     0.214147  
 -0.0891686  -0.111353    -0.133395   -0.147858      -0.116359    -0.126466  
 -0.131059   -0.0347357   -0.0690151  -0.0121648      0.00245444  -0.0239689 
 -0.170803   -0.143918    -0.142024   -0.195945      -0.196819    -0.167209  

# Question 2